# 2021/03/11 Study Session Questions

## 1. Reverse String

## Problem
Write a function reverse_words() that takes a message as a list of characters and reverses the order of the words in place.

> Why a list of characters instead of a string?

The goal of this question is to practice manipulating strings in place. Since we're modifying the message, we need a mutable ↴ type like a list, instead of Python 3.6's immutable strings.

## Observation
1. I need to visit every element at least once to make sure everything flips
2. In place would require no additional memory in regards to n

-> Best case for the complexities are `O(N)` for time and `O(1)` for space

## Approach
With Python magic, slicing will automatically give me what I want. `arr[start:end:step]` syntax shows that if I fo `[None:None:-1]` it will reversely traverse an entire list

In [1]:
def reverseWords(message: [str]) -> str:
    if not message: return ''
    # can just return message[::-1], python is smart
    return ''.join(message[::-1])

In [2]:
# Test
targetMsg = 'hello world!'
testMsg = ['!', 'd', 'l', 'r', 'o', 'w', ' ', 'o', 'l', 'l', 'e', 'h']
reverseWords(testMsg)

'hello world!'

## Non-Slicing Solution
Slicing makes a copy of the existing array. How can we save this space?

### Buffer and Replace
To do this in constant space, I can do the following
1. Start at the begining of the list, and assign two pointers one at the start of the array and one at the end of the array.
2. Place `arr[0]` on the buffer
3. copy over what's at `arr[len(arr)-1]` to `arr[0]`
4. place what's in the buffer to `arr[len(arr)]`
5. decrement end pointer and increment the start pointer
6. repeat this until the two pointers cross.

In [3]:
# Buffer and Replace
def bufferAndReplace(message: [str]) -> str:
    if not message: return ''
    
    # ptrs and buffers
    start, end, buffer = 0, len(message)-1, ''
    
    # 6. repeat this until the two pointers cross
    while start < end:
        # 2. Place `arr[0]` on the buffer
        buffer = message[start]
        # 3. copy over what's at `arr[len(arr)-1]` to `arr[0]`
        message[start] = message[end]
        # 4. place what's in the buffer to `arr[len(arr)]`
        message[end] = buffer
        # 5. decrement end pointer and increment the start pointer
        end -= 1
        start += 1
    
    return ''.join(message)

In [4]:
# Test
targetMsg = 'hello world!'
testMsg = ['!', 'd', 'l', 'r', 'o', 'w', ' ', 'o', 'l', 'l', 'e', 'h']
bufferAndReplace(testMsg)

'hello world!'

## Complexity

### Time
- Array Slicing in Python is `O(|i2 - i1|)` where `i2` is the ending index and `i1` is the starting index. In this case we need to visit every element in the array, so `O(n)`
- Buffer and replace will go through each element once, thus `O(N)`

*In both cases, we are ignoring the cost for joining the array.*

### Space
- List Slicing in Python has the same complexity as `O(N)` because slicing will create a copy and take the size of slice.
- Buffer and replace will only take 3 more location in memory because it stores 2 pointers for the start and end pointer, and one to store char in the buffer. `O(3)` addtional memory is constant space, `O(1)`

## 2.Merge Array Sorted

## Problem
Each order is represented by an "order id" (an integer).
We have our lists of orders sorted numerically already, in lists. Write a function to merge our lists of orders into one sorted list.
For example:
```python
my_list     = [3, 4, 6, 10, 11, 15]
alices_list = [1, 5, 8, 12, 14, 19]

# Prints [1, 3, 4, 5, 6, 8, 10, 11, 12, 14, 15, 19]
print(merge_lists(my_list, alices_list))
```

### Observation
1. Two lists contain only positive integers
2. Two lists are sorted in their own order.
3. Two lists are not necessarily of the same length
4. I need to visit every element in both lists at least once to merge them completely

### Approach

#### Brute Force: Merge and Sort
I can merge two lists first, then sort the merged list. Assuming we use quicksort, the complexity of this operation becomes O(n) + O(m) + O((n+m)log(n+m)), and the overall complexity will be capped by O((n+m)log(n+m)). Space complexity is O(m) because we will need to increase the size of the larger array by the size of the smaller array in order to append the two.

1. merge the two lists
2. sort the merged lists

As we can assume that the total number of elements to go through would be n+m, we can reasonably assume that this will be O((2n)log(2n)) which will be O(nlogn).

The bottle neck here is simply the complexity of the sort. How do we avoid sorting the merged list?

#### two pointer comparison

The least time complexity has to be O(n+m), where n and m are respective length of the lists. The length of two lists are not guaranteed to be the same.
As a human, I would do the following

1. Check the first elements of one array to another
2. Place the smaller element in a new, merged array
3. For the array where you took the element from, move down one number. 
4. Make the same comparison to the new element from array 1 and the current element form array 2
4. Do this until one of the array is fully traversed
5. Append what remains of the array with leftovers to the merged array list

In [5]:
def mergeThenSort(arr1: [int], arr2:[int]) -> [int]:
    mergedArr = arr1+arr2
    mergedArr.sort()
    return mergedArr

In [6]:
def mergeTwoPtrCmp(arr1: [int], arr2: [int]) -> [int]:
        if not arr1 or not arr2: return arr1+arr2

        ptr1, ptr2, mergedArr, len1, len2 = 0, 0, [], len(arr1), len(arr2)
        # 5. Do this until one of the array is fully traversed
        while ptr1 < len1 and ptr2 < len2:
            # 1. Check the first element of one array to another
            if arr1[ptr1] <= arr2[ptr2]:
                # 2. Place the smaller element in a new, merged array
                mergedArr.append(arr1[ptr1])
                # 3. For the array where you took the element from, move down one number. 
                ptr1 += 1
            else:
                # 2. Place the smaller element in a new, merged array
                mergedArr.append(arr2[ptr2])
                # 3. For the array where you took the element from, move down one number. 
                ptr2 += 1
        # 5. Append what remains of the array with leftovers to the merged array list
        if ptr1 < len1:
            for n in arr1[ptr1:]:
                mergedArr.append(n)
        else:
            for n in arr2[ptr2:]:
                mergedArr.append(n)
        
        return mergedArr

In [7]:
# test
arr1 = [1,3,5]
arr2 = [2,4,6]
res  = [1,2,3,4,5,6]

mergeTwoPtrCmp(arr1,arr2)

[1, 2, 3, 4, 5, 6]

In [8]:
# test
arr1 = [1,3,5]
arr2 = [2,4,6]
res  = [1,2,3,4,5,6]

mergeThenSort(arr1, arr2)

[1, 2, 3, 4, 5, 6]

## Complexity

### Time
O(n+m) -> traverse both arrays

### Space
O(n+m) -> Additional space to copy over the full two arrays